# Coursera Data Science Capstone

#### Notebook used for the Final Project of the IBM Data Science Professional Certificate

#### Import the required libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
!pip install geocoder
import geocoder
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

#### Download the data and format it

In [40]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

In [41]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [42]:
neighborhoods_data = newyork_data['features']

#### Create a dataframe and a function to fill the data into the dataframe

In [43]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [44]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [54]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [99]:
# NY_data = neighborhoods.loc[neighborhoods['Borough'].isin(['Brooklyn','Queens'])].reset_index(drop=True)
NY_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)

#### Using geopy library to get the latitude and longitude values of New York City

In [100]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Creating a map of New York with neighborhoods

In [101]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(NY_data['Latitude'], NY_data['Longitude'], NY_data['Borough'], NY_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Using the Foursquare API

#### We are going to start using the Foursquare API to explore the neighborhoods and segment them
First we define our credentials (kept secret here)

In [104]:
# @hidden-cell
CLIENT_ID = 'WDMQ2G40J0ZUTBBIVTMJ5WHFEUYDSR3FSLQML0WK2CSBXOQB' # your Foursquare ID
CLIENT_SECRET = 'UBR55TPVIDRQ4RDO0IB320B3M0512K0NG1MUSJ3R2HBVRPAE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WDMQ2G40J0ZUTBBIVTMJ5WHFEUYDSR3FSLQML0WK2CSBXOQB
CLIENT_SECRET:UBR55TPVIDRQ4RDO0IB320B3M0512K0NG1MUSJ3R2HBVRPAE


In [152]:
#NEWWWWWW
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # your Foursquare ID
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY
CLIENT_SECRET:1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B


In [103]:
#NEWWWWWW 222222222
CLIENT_ID = 'HPTTXE151PAZP3JOZ4JXGC5KK3ABHC4K3TBDFXS0XCHFH5VW' # your Foursquare ID
CLIENT_SECRET = 'V4QC5NTCQJTKAVG2QRJ2ZQLQ4X2XLPJLC22GKWGRXJCBFEK5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HPTTXE151PAZP3JOZ4JXGC5KK3ABHC4K3TBDFXS0XCHFH5VW
CLIENT_SECRET:V4QC5NTCQJTKAVG2QRJ2ZQLQ4X2XLPJLC22GKWGRXJCBFEK5


#### Function that extracts the category of the venue

In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=350):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe called  _NY_venues_

In [105]:
NY_venues = getNearbyVenues(names=NY_data['Neighborhood'],
                                   latitudes=NY_data['Latitude'],
                                   longitudes=NY_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


#### Size of the resulting dataframe and how many unique categories there are

In [106]:
print(NY_venues.shape)
NY_venues.head()

(1609, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
2,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
3,Bay Ridge,40.625801,-74.030621,Georgian Dream Cafe and Bakery,40.625586,-74.030196,Caucasian Restaurant
4,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar


In [107]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 249 uniques categories.


### Analyzing neighborhoods

In [108]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lawyer,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Pop-Up Shop,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### New dataframe size

In [109]:
NY_onehot.shape

(1609, 249)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [110]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
# NY_grouped

In [111]:
NY_grouped.shape

(70, 249)

#### Print each neighborhood along with the top 5 most common venues

In [112]:
num_top_venues = 5

for hood in NY_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0  Italian Restaurant  0.14
1         Gas Station  0.14
2            Pharmacy  0.14
3          Restaurant  0.07
4          Donut Shop  0.07


----Bay Ridge----
                venue  freq
0  Italian Restaurant  0.14
1      Ice Cream Shop  0.05
2       Grocery Store  0.05
3    Greek Restaurant  0.05
4                 Spa  0.05


----Bedford Stuyvesant----
                venue  freq
0         Coffee Shop  0.22
1            Boutique  0.11
2      Discount Store  0.11
3  Italian Restaurant  0.11
4           Juice Bar  0.11


----Bensonhurst----
                venue  freq
0  Italian Restaurant  0.29
1        Liquor Store  0.14
2  Chinese Restaurant  0.14
3           Pet Store  0.14
4        Dance Studio  0.14


----Bergen Beach----
                           venue  freq
0                 Baseball Field   1.0
1  Paper / Office Supplies Store   0.0
2                         Museum   0.0
3                    Music Store   0.0
4                   

#### Put everything into a new dataframe

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Gas Station,Italian Restaurant,Pharmacy,Hookah Bar,Ice Cream Shop,Pizza Place,Flower Shop,Donut Shop,Restaurant,Bagel Shop
1,Bay Ridge,Italian Restaurant,Spa,Grocery Store,Greek Restaurant,Ice Cream Shop,Pizza Place,Bar,Yoga Studio,Supermarket,Pool Hall
2,Bedford Stuyvesant,Coffee Shop,Juice Bar,Pizza Place,Discount Store,Italian Restaurant,Bar,Café,Boutique,Fish & Chips Shop,Fast Food Restaurant
3,Bensonhurst,Italian Restaurant,Pet Store,Liquor Store,Pizza Place,Dance Studio,Chinese Restaurant,Women's Store,Farm,Farmers Market,Fast Food Restaurant
4,Bergen Beach,Baseball Field,Women's Store,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant


In [119]:
#NY_data = neighborhoods.loc[neighborhoods['Borough'].isin(['Brooklyn','Queens'])].reset_index(drop=True)
NEW_NEI = neighborhoods_venues_sorted

NEW_NEI = NEW_NEI.loc[(NEW_NEI['1st Most Common Venue'] != 'Bakery') & (NEW_NEI['1st Most Common Venue'] != 'Donut Shop') & (NEW_NEI['1st Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['1st Most Common Venue'] != 'Café') & (NEW_NEI['1st Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['2nd Most Common Venue'] != 'Bakery') & (NEW_NEI['2nd Most Common Venue'] != 'Donut Shop') & (NEW_NEI['2nd Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['2nd Most Common Venue'] != 'Café') & (NEW_NEI['2nd Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['3rd Most Common Venue'] != 'Bakery') & (NEW_NEI['3rd Most Common Venue'] != 'Donut Shop') & (NEW_NEI['3rd Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['3rd Most Common Venue'] != 'Café') & (NEW_NEI['3rd Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['4th Most Common Venue'] != 'Bakery') & (NEW_NEI['4th Most Common Venue'] != 'Donut Shop') & (NEW_NEI['4th Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['4th Most Common Venue'] != 'Café') & (NEW_NEI['4th Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['5th Most Common Venue'] != 'Bakery') & (NEW_NEI['5th Most Common Venue'] != 'Donut Shop') & (NEW_NEI['5th Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['5th Most Common Venue'] != 'Café') & (NEW_NEI['5th Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['6th Most Common Venue'] != 'Bakery') & (NEW_NEI['6th Most Common Venue'] != 'Donut Shop') & (NEW_NEI['6th Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['6th Most Common Venue'] != 'Café') & (NEW_NEI['6th Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['7th Most Common Venue'] != 'Bakery') & (NEW_NEI['7th Most Common Venue'] != 'Donut Shop') & (NEW_NEI['7th Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['7th Most Common Venue'] != 'Café') & (NEW_NEI['7th Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['8th Most Common Venue'] != 'Bakery') & (NEW_NEI['8th Most Common Venue'] != 'Donut Shop') & (NEW_NEI['8th Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['8th Most Common Venue'] != 'Café') & (NEW_NEI['8th Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['9th Most Common Venue'] != 'Bakery') & (NEW_NEI['9th Most Common Venue'] != 'Donut Shop') & (NEW_NEI['9th Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['9th Most Common Venue'] != 'Café') & (NEW_NEI['9th Most Common Venue'] != 'Coffe Shop')]
NEW_NEI = NEW_NEI.loc[(NEW_NEI['10th Most Common Venue'] != 'Bakery') & (NEW_NEI['10th Most Common Venue'] != 'Donut Shop') & (NEW_NEI['10th Most Common Venue'] != 'Bagel Shop') & (NEW_NEI['10th Most Common Venue'] != 'Café') & (NEW_NEI['10th Most Common Venue'] != 'Coffe Shop')].reset_index(drop=True)

#NEW_NEI = neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['1st Most Common Venue'].isin(['Bakery','Beach'])].reset_index(drop=True)
#NEW_NEI = NEW_NEI.loc[NEW_NEI['2nd Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['3rd Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['4th Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['5th Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['6th Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['7th Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['8th Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['9th Most Common Venue'] != 'Bakery']
#NEW_NEI = NEW_NEI.loc[NEW_NEI['10th Most Common Venue'] != 'Bakery'].reset_index(drop=True)
NEW_NEI


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Spa,Grocery Store,Greek Restaurant,Ice Cream Shop,Pizza Place,Bar,Yoga Studio,Supermarket,Pool Hall
1,Bensonhurst,Italian Restaurant,Pet Store,Liquor Store,Pizza Place,Dance Studio,Chinese Restaurant,Women's Store,Farm,Farmers Market,Fast Food Restaurant
2,Bergen Beach,Baseball Field,Women's Store,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
3,Boerum Hill,Bar,Furniture / Home Store,Yoga Studio,Spa,Coffee Shop,Sandwich Place,Boxing Gym,Cajun / Creole Restaurant,Paper / Office Supplies Store,Middle Eastern Restaurant
4,Brighton Beach,Eastern European Restaurant,Gourmet Shop,Sushi Restaurant,Restaurant,Other Great Outdoors,Bookstore,Russian Restaurant,Korean Restaurant,Coffee Shop,Bank
5,Broadway Junction,Sandwich Place,Dessert Shop,Fried Chicken Joint,Dry Cleaner,Diner,Hotel,Deli / Bodega,Caribbean Restaurant,Moving Target,Business Service
6,Brooklyn Heights,Deli / Bodega,Yoga Studio,Cosmetics Shop,Plaza,Pizza Place,Pharmacy,Pet Store,Mexican Restaurant,Ice Cream Shop,Grocery Store
7,Brownsville,Spanish Restaurant,Fried Chicken Joint,Farmers Market,Trail,Park,Convenience Store,Burger Joint,Pizza Place,Flea Market,Fish Market
8,Bushwick,Mexican Restaurant,Coffee Shop,Bar,Deli / Bodega,Thrift / Vintage Store,Mediterranean Restaurant,Nightclub,Discount Store,French Restaurant,Pizza Place
9,Canarsie,Lawyer,Asian Restaurant,Grocery Store,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Event Space
